![image info](https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/images/banner_1.png)

# Proyecto 1 - Predicción de precios de vehículos usados

En este proyecto podrán poner en práctica sus conocimientos sobre modelos predictivos basados en árboles y ensambles, y sobre la disponibilización de modelos. Para su desasrrollo tengan en cuenta las instrucciones dadas en la "Guía del proyecto 1: Predicción de precios de vehículos usados".

**Entrega**: La entrega del proyecto deberán realizarla durante la semana 4. Sin embargo, es importante que avancen en la semana 3 en el modelado del problema y en parte del informe, tal y como se les indicó en la guía.

Para hacer la entrega, deberán adjuntar el informe autocontenido en PDF a la actividad de entrega del proyecto que encontrarán en la semana 4, y subir el archivo de predicciones a la [competencia de Kaggle](https://www.kaggle.com/competitions/miad2024-12-prediccion-precio-vehiculos).

## Datos para la predicción de precios de vehículos usados

En este proyecto se usará el conjunto de datos de Car Listings de Kaggle, donde cada observación representa el precio de un automóvil teniendo en cuenta distintas variables como: año, marca, modelo, entre otras. El objetivo es predecir el precio del automóvil. Para más detalles puede visitar el siguiente enlace: [datos](https://www.kaggle.com/jpayne/852k-used-car-listings).

## Ejemplo predicción conjunto de test para envío a Kaggle

En esta sección encontrarán el formato en el que deben guardar los resultados de la predicción para que puedan subirlos a la competencia en Kaggle.

In [1]:
import warnings
warnings.filterwarnings('ignore')
!python --version

Python 3.10.5


In [2]:
# Importación librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
# Librerías para Procesamiento de datos
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
# Librerías para Modelos
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import joblib

In [3]:
# Carga de datos de archivo .csv
data = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/datasets/dataTrain_carListings.zip')

In [4]:
# Procesamiento de datos
data['Mileage'] = np.log(data['Mileage'] + 1)  # Transformación de Mileage

# Encoding de variables categóricas
le = LabelEncoder()
for col in ['State', 'Make', 'Model']:
    data[col] = le.fit_transform(data[col])

# Preparación de variables
X = data[['Year', 'Mileage', 'State', 'Make', 'Model']]
y = np.log(data['Price'])  # Transformación de Price

# División del dataset en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Entrenamiento del modelo Random Forest
model = RandomForestRegressor(max_features=4,n_estimators=100, max_depth=19, random_state=1, min_samples_split=17, n_jobs=-1)
model.fit(X_train, y_train)

# Guardar el modelo entrenado
joblib.dump(model, 'model_rf.pk1', compress=9)

['model_rf.pk1']

API

In [6]:
from flask import Flask, request, jsonify
from flask_restx import Api, Resource, fields
import joblib

app = Flask(__name__)
api = Api(app, version='1.0', title='Predicción de precios de vehículos usados',
          description='API usa Random Forest para predecir precios de vehiculos')

ns = api.namespace('predict', description='Price Predictions')

model = joblib.load('model_rf.pkl')  # Cargar el modelo guardado

# Modelo de entrada de datos para la API
predict_request = api.model('PredictRequest', {
    'Year': fields.Integer(required=True, description='Year of the vehicle'),
    'Mileage': fields.Float(required=True, description='Logged mileage of the vehicle'),
    'State': fields.Integer(required=True, description='Encoded state'),
    'Make': fields.Integer(required=True, description='Encoded make'),
    'Model': fields.Integer(required=True, description='Encoded model')
})

@ns.route('/')
class PricePrediction(Resource):
    @ns.expect(predict_request)
    def post(self):
        # Recibe datos de entrada para predicción
        data = request.json
        X_new = [data['Year'], np.log(data['Mileage'] + 1), data['State'], data['Make'], data['Model']]
        # Realizar predicción
        y_pred = model.predict([X_new])
        # Devolver resultado
        return jsonify({
            'price': np.exp(y_pred[0])  # Transformación inversa del logaritmo
        })

if __name__ == '__main__':
    app.run(debug=True)

FileNotFoundError: [Errno 2] No such file or directory: 'model_rf.pkl'